In [18]:
from __future__ import print_function
import warnings

## Exercise 5: Calendar events

We want to keep a schedule of events.  We will do this by creating a class called `Day`.  It is sketched out below.  A `Day` holds a list of events and has methods that allow you to add an delete events.  Our events will be instances of a class `Event`, which holds the time, location, and description of the event.

Finally, we can keep track of a list of all the `Day`s for which we have events to make our schedule.

Fill in these classes and write some code to demonstrate their use:

  * Create a full week of days in your calendar
  * Add an event every day at noon called "lunch"
  * Randomly add some other events to fill out your calendar
  * Write some code that tells you the start time of your first meeting and the end time of your last meeting (this is the length of your work day)

---

### My solution:
I went a little overboard here ... I'm sorry it's a lot to read! I realized towards the end that it's very long and no one likes to read code, so I tried to make it as easy to follow as possible. I had a lot of fun writing everything, though!

It's mostly comments (to make reading easier) and docstrings (to get into good habits). After that, it's mostly checking the arguments to the functions to make sure they are valid.

Only a small portion of the code actually produces some kind of output (the output asked for above). 

For each class, I tried to summarize the important methods. That should give an idea of what each class can do. 

Each method has a docstring you can refer to for more information about its arguments and output. I also tried to add many comments to explain my reasoning or simply what each block of code is doing.

---

#### The `Event` class

The class `Event` is filled out below. I added the following, in addition to what you've provided:

- An optional `description` argument (this was included in the instructions but not the class itself)


- Check the arguments to `__init__()`, raising an error if any are not valid.
    
    - (I've also done this for the `Day` and `Week` classes below, along with some other class methods).


- A function `hours_and_minutes()` that takes a `time` (`int` or `float`) and returns integers for the hours and minutes.

    - For example, `hours_and_minutes(13.5)` returns `13` hours and `30` minutes. This could be formatted as a time, `13:30` for 1:30PM.
    
    - This makes it easier to compare starting times of events using integers (see below).


- `__gt__()`, `__lt__()`, and `__eq__()` methods to compare two events.

    - For example, if we initialize two `Event` objects, `event1` and `event2`, then:
        - `event1` begins after `event2` if `event1 > event2` is `True`.
        - `event1` begins before `event2` if `event1 < event2` is `True`.
        - They begin at the same time if `event1 == event2` is `True`.
         
    - __How it works__: the user provides the starting time, `time` (in hours), which is then converted to hours and minutes. Then I compare the value of `event1.time_hr*60 + event1.time_min` with `event2.time_hr*60 + event2.time_min` (_i.e._, event starting time, in minutes since midnight). The point here is that we are comparing integers, and the event that starts later will have a larger value.
        
    - This is helpful when sorting events in the `Day` class to print out a daily schedule.

In [19]:
class Event:
    """a single event in our calendar"""
    
    def __init__(self, name, time=9, location=None, duration=1, description=None):
        """Create an event.
        
        Parameters
        ----------
        name : str
            A name for the event.
        time : int or float
            The time at which the event begins, in hours from midnight.
        location : str, default=None
            The location of the event. If not provided, `location` is not used.
        duration : int or float
            The duration of the event, in hours.
        description: str, default=None
            A description of the event. If not provided, `description` is not used.
            
        Raises
        ------
        ValueError
            If `time` is less than zero or greater than 24 hours,
        NotImplementedError
            If `duration` is a negative value,
            or if `time` + `duration` > 24 hours (the event ends after midnight).
        TypeError
            If any of the parameters do not have the required type listed above.
            
        Notes
        -----
        All times are in 24-hour format. For fractional times a float should be used. 
        For example, `time` = 13.5 corresponds to 1:30 PM, 
        and `duration`=1.25 corresponds to a duration of 1 hour and 15 minutes.
        
        Currently, you cannot add an event that continues into the next day (i.e., ends after midnight)
        """
        # ------------ verify the inputs ------------------------
        
        # check the values for the time
        if (float(time) < 0.0) or (float(time) > 24.0):
            raise(ValueError("The hour of the starting time must be between 0.0 and 24.0"))
        if (float(duration) < 0.0):
            raise(NotImplementedError("The duration cannot be negative."))
            
        # check the argument types and if ok, set the class attributes
        valid_time_types = [int, float]
        
        if type(name) is not str:
            raise(TypeError("'name' must be a str."))
        else:
            self.name = name
        
        if type(time) not in valid_time_types:
            raise(TypeError("'time' must be an int or a float."))
        else:
            self.time = time
            
        if (location is not None) and (type(location) is not str):
            raise(TypeError("'location' must be a str."))
        else:
            self.location = location
            
        if type(duration) not in valid_time_types:
            raise(TypeError("'duration' must be an int or a float."))
        else:
            self.duration = duration
            
        if (description is not None) and (type(description) is not str):
            raise(TypeError("'description' must be a str."))
        else:
            self.description = description
        
        # ------------ initialize the event --------------------------
        
        # use hours and minutes for the time - useful to compare times between events
        self.time_hr, self.time_min = self.hours_and_minutes(self.time)
        self.duration_hr, self.duration_min = self.hours_and_minutes(self.duration)
        
        # also define the end time of the event (helps to avoid conflicts between events)
        self.end = self.time + self.duration
        if float(self.end) > 24.0: # the event runs into the next day
            raise(NotImplementedError("Cannot handle an event that continues into the next day (ends after midnight)."))
        self.end_hr, self.end_min = self.hours_and_minutes(self.end)


    def hours_and_minutes(self, time):
        """Get the time in hours and minutes.

        Parameters
        ----------
        time : int or float
            The time, in hours.

        Returns
        -------
        hours : int
            An integer giving the number of hours in `time`.
        minutes : int
            An integer giving the number of minutes left over in `time` - `hours`.
        """
        hours = int(time)
        minutes = time - hours # the leftover time, in hours
        minutes *= 60.0 # convert from hours to minutes
        minutes = int(minutes) # round down - we can neglect a few seconds for simplicity
        return hours, minutes

    
    def check(self, other):
        """Check if the argument `other` is also an `Event` object.

        Parameters
        ----------
        other : Event
            An `Event` object.

        Raises
        ------
        ValueError
            When `other` is not an `Event` object.
        """
        if not isinstance(other, Event):
            raise(ValueError("{} is not an Event object.".format(other)))


    def __lt__(self, other):
        """Compare the starting time of two events to see which begins earlier.

        Parameters
        ----------
        other : Event
            An instance of the `Event` class.

        Returns
        -------
        bool
            True if `self.time` is earlier than `other.time`.  
        """
        self.check(other)
        return (self.time_hr*60 + self.time_min) < (other.time_hr*60 + other.time_min)


    def __gt__(self, other):
        """Compare the starting time of two events to see which begins later.

        Parameters
        ----------
        other : Event
            An instance of the `Event` class.

        Returns
        -------
        bool
            True if `self.time` is later than `other.time`.  
        """
        self.check(other)
        return (self.time_hr*60 + self.time_min) > (other.time_hr*60 + other.time_min)


    def __eq__(self, other):
        """Compare the starting time of two events to see if they begin at the same time.

        Parameters
        ----------
        other : Event
            An instance of the `Event` class.

        Returns
        -------
        bool
            True if `self.time` is the same as `other.time`.  
        """
        self.check(other)
        return (self.time_hr*60 + self.time_min) == (other.time_hr*60 + other.time_min)

Test the `Event` class by creating some fake events:

In [11]:
event1 = Event('first event', time=6, duration=1)
event2 = Event('second event', time=12, duration=1)
event3 = Event('third event', time=18, duration=1)
event4 = Event('fourth event', time=6, duration=1)

What I'm really interested in is testing the comparison between events:

In [12]:
event1 < event2

True

In [13]:
event3 > event2

True

In [14]:
event4 == event1

True

In [15]:
event2 < event4

False

---
#### The `Day` class

In the `Day` class, there is an optional argument to `__init__()`, in addition to the date:

- `allow_conflict`: If `True`, events that overlap are allowed and a warning will be raised by `add_event()` when trying to add the  conflicting event. If the value is `False`, an error will be raised instead.
    - Default is `True`.
    
    
- The other (required) arguments are still `month`, `day`, and `year`, which should all be integers. For `month`, `1` is January, `2` is February, etc.



There are three __"helper" methods__ (but they could also be useful on their own):


- `sort_events_by_time()` takes the current list of events, and returns a new list of events sorted by their start time, with the earliest event first.


- `new_event_conflicts()` is called by `add_event()` when adding a new event to your day. It checks if the new event conflicts with any currently scheduled event. 
    - The input `new_event` is an instance of the `Event` class.
    - Returns a list of events (objects of the `Event` class) that conflict with the new event.
    - In particular, this function checks, for each `event` in the list of current events, whether:
        - The `new_event` begins during the `event`.
        - The `new_event` ends during the `event`.
        - The `new_event` begins before the `event` and ends after it. 


- `length_of_day()` returns the length of the day, in hours, from the beginning of the first event to the end of the last event.



The __main methods__ are:

- `add_event()`: Add a new event to the list for the day.
    - Takes in same arguments as `__init__()` in the `Event` class.
    - Checks if there are conflicts by calling `new_event_conflicts()`, and then takes the appropriate action:
        - If `allow_conflicts = True`, the user is warned and the event is added.
        - If `allow_conflicts = False`, an error is raised.


- `delete_event()`: Delete an event.
    - Takes in the `name` of the event as an argument.
    - Optional argument `confirm`: if `True`, the user is asked to confirm that they would really like to delete the event.
 
 
- `print_schedule()`: Print out the schedule for the day, in order from the first event to the last.
    - Calls `sort_events_by_time()` to get the events in order.
    - Optional argument `print_day` (default is `True`) will also print the month, day, and year at the top of the schedule.

In [27]:
class Day:
    """a single day keeping track of the events scheduled"""
    
    def __init__(self, month, day, year, allow_conflicts=True):
        """Begin a new day!
        
        Parameters
        ----------
        month : int
            The month of the day being planned.
        day : int
            The day of the month (calendar date).
        year : int
            The year of the day being planned.
        allow_conflicts : bool, default=True
            If True, events that overlap are allowed, and a warning will be raised by `add_event()` when
             trying to add the conflicting event.
            Otherwise, an error will be raised by `add_event()`.
            
        Raises
        ------
        TypeError
            If any of the above parameters have the incorrect type.
            
        Notes
        -----
        `month`, `day`, and `year` are allowed to be a float, but will be converted to an int.
        
        The user is responsible for ensuring that the `month`, `day`, and `year` are correct.
        For example, this code will gladly accept `month`=100.
        """
        # ----------- check the arguments -----------------
        valid_date_types = [int, float]
        # for the following three variables, will be True if the corresponding argument is not a number
        bad_month = type(month) not in valid_date_types
        bad_day = type(day) not in valid_date_types 
        bad_year = type(year) not in valid_date_types
        if bad_month or bad_year or bad_day:
            raise(TypeError("'month', 'day', and 'year' must each be an int or a float."))
        # check the optional argument
        if type(allow_conflicts) is not bool:
            raise(TypeError("'allow_conflicts' must be True or False."))
        
        # ------------ initialize the day -----------------
        # set the date
        self.month = int(month)
        self.day = int(day)
        self.year = int(year)
        
        # keep track of the events
        self.allow_conflicts = allow_conflicts
        self.events = []
    
    
    def sort_events_by_time(self):
        """Sort the current list of events by their starting time.
        
        Returns
        -------
        list
            A list of the events, sorted by their starting time, beginning with the first event.
        """
        copy_of_events = self.events[:] # we don't want to change the original copy
        copy_of_events.sort() # the sort() method should work on the list of Event objects
        return copy_of_events[:] # return a copy, just to be safe
    

    def new_event_conflicts(self, new_event):
        """Check if the new event conflicts with any events that are already scheduled.
        
        Parameters
        ----------
        new_event : Event
            The new event to be added to the schedule.
            
        Returns
        -------
        conflicts : list
            A list of events which conflict with the new event.
        """
        conflicts = [] # list to hold events that conflict with the new event
        
        # loop trough the list of events, sorted by start time, and check for conflicts
        for event in self.sort_events_by_time():
            if (new_event.time > event.time) and (new_event.time < event.end):
                # the new_event begins during this event
                conflicts.append(event)
            elif (new_event.end > event.time) and (new_event.end < event.end):
                # the new_event ends during this event
                conflicts.append(event)
            elif (new_event.time < event.time) and (new_event.end > event.end):
                # the new_event begins before this event starts and ends after this event ends.
                #  in other words, this event occurs while new_event is already occuring
                conflicts.append(event)
        
        return conflicts
    
    
    def length_of_day(self):
        """Get the length of the day, in hours, based on the currently scheduled events.
        
        Returns
        -------
        length : int or float
            The length of the day, from the beginning of the first event to the end
            of the last event, in hours.
        """
        ordered_events = self.sort_events_by_time()
        # find the starting time of the first event
        start = ordered_events[0].time
        # events have varying durations, so just get a list of the end times and find the max value
        end_times = [event.end for event in self.events]
        end = max(end_times)
        # now just find the difference
        length = end - start
        return length
    
    
    def add_event(self, name, time=9, location=None, duration=1, description=None):
        """Add a new instance of the `Event` class to the list of events for the day,
        after checking for conflicts with any currently scheduled events.
        
        Parameters
        ----------
        name : str
            A name for the event.
        time : int or float
            The time at which the event begins, in hours from midnight.
        location : str, default=None
            The location of the event. If not provided, `location` is not used.
        duration : int or float
            The duration of the event, in hours.
        description: str, default=None
            A description of the event. If not provided, `description` is not used.
            
        Raises
        ------
        RuntimeError
            If `allow_conflicts`=False and the new event conflicts with an existing event.
            
        Notes
        -----
        All times are in 24-hour format. For fractional times a float should be used. 
        For example, `time` = 13.5 corresponds to 1:30 PM, 
        and `duration`=1.25 corresponds to a duration of 1 hour and 15 minutes.
        """
        # create the new event
        new_event = Event(name, time=time, location=location, duration=duration, description=description)
        
        # check if there are any conflicts with existing events
        event_conflicts = self.new_event_conflicts(new_event) # list of events that conflict with new one
        num_conflicts = len(event_conflicts)
        if num_conflicts > 0:
            # create message to inform user of conflicts, via warning or error, depending on user choice
            message = "You have set allow_conflicts={}.".format(self.allow_conflicts)
            message +=  "\nThe new event '{}' conflicts with the following {} existing event(s):".format(name, num_conflicts)
            for event in event_conflicts:
                message += "\n{}".format(event.name)
            # decide what to do 
            if self.allow_conflicts: # just warn the user
                warnings.warn(message)
            else: # raise an error and terminate
                raise(RuntimeError(message))
                
        # if no conflicts, or if we don't care about conflicts, add the event
        self.events.append(new_event)
    
    
    def delete_event(self, name, confirm=True):
        """Remove the instance of the `Event` class with `name` from the list 
        of events for the day, after confirmation from the user.
        
        Parameters
        ----------
        name : str
            A name for the event.
        confirm : bool, default=True
            If True, ask the user if they are sure they would like 
            to delete the event called `name` before proceeding.
            
        Warns
        -----
        UserWarning
            If the event does not exist, warn the user that nothing is being done.
        """
        # find the index of the event in the list by using its name
        event_names = [event.name for event in self.events] # should be in same order as self.events
        try: # if the event exists, get its index in the list
            idx = event_names.index(name)
        except ValueError: # if not, warn the user
            warnings.warn("The event named '{}' does not exist, so nothing further will be done.".format(name))
            return None # exit the function
        
        # if the event does exist, confirm the deletion with the user first (if they asked you to do so)
        if confirm: 
            valid_inputs = ['y', 'n'] # we're asking a yes or no question
            invalid_input = True      # assume they don't know what they want!
            while(invalid_input):     # keep asking until they get it right
                print("Are you sure that you want to delete the event named '{}'?".format(name))
                user_input = input("Enter 'yes' or 'no': ")
                user_input = user_input.lower()[0]
                if user_input in valid_inputs: # then don't ask then again
                    invalid_input = False
                    if user_input == 'n': # they changed their mind, so don't delete the event
                        print("The event named '{}' will NOT be deleted.".format(name))
                        return None # exit without deleting
        
        # now we're really sure we want to delete the event, so let's do it
        self.events.pop(idx)
        return None # for consistency
    
    
    def print_schedule(self, print_day=True):
        """Print out the schedule for the day, in order, beginning with the earliest event.
        
        Parameters
        ----------
        print_day : bool, default=True
            If True, will print the year, month, and date of the current day before printing the schedule.
            Otherwise, it will just print the schedule.
            
        Notes
        -----
        Nearly everything here is just to make the output look nicer, so I didn't add too many comments.
        """
        # templates for printing out the information about each event
        event_name = "\nEvent name: {}"
        event_start = "\nStarts at {:d}:{:02d}"
        event_end = "\nEnds at {:d}:{:02d}"
        event_duration = "\nDuration: {} hours, {} minutes" 
        event_location = "\nLocation: {}"
        event_description = "\nDescription: {}"
        event_conflicts = "\nConflicts with: {}"
        day_length = "\nLength of day: {:1.2f} hours.".format(self.length_of_day())
        divider = '-' * 80
        
        if print_day:
            print("Year: {}, Month: {}, Day: {}".format(self.year, self.month, self.day))
            print(divider)
        
        # loop through events in order, beginning at earliest
        for event in self.sort_events_by_time():
            # look for conflicts and print them out if there are any
            conflicts = self.new_event_conflicts(event)
            num_conflicts = len(conflicts)
            if num_conflicts > 0:
                conflict_msg = ""
                for conflict in conflicts:
                    conflict_msg += "\n{}".format(conflict.name)
            
            # print out the information
            print(event_name.format(event.name))
            if num_conflicts > 0:
                print(event_conflicts.format(conflict_msg))
            print(event_start.format(event.time_hr, event.time_min))
            print(event_end.format(event.end_hr, event.end_min))
            print(event_duration.format(event.duration_hr, event.duration_min))
            if event.location is not None:
                print(event_location.format(event.location))
            if event.description is not None:
                print(event_description.format(event.description)) 
            print(divider)
            
        print(day_length)
        print(divider)

Test the `Day` class:

In [28]:
# create a new day
today = Day(month=3, day=15, year=2021)

# add some events
today.add_event("lunch", time=12, duration=1, location="home", description="food")

today.add_event("dinner", time=18, duration=1, location="home", description="more food")

today.add_event("snack", time=15, duration=1, location="home", description="even more food")

today.add_event("meeting", time=12, duration=0.5, location="zoom", description="while I'm eating food")

today.add_event("breakfast", time=9, duration=1, location="home", description="morning food")

today.print_schedule()

Year: 2021, Month: 3, Day: 15
--------------------------------------------------------------------------------

Event name: breakfast

Starts at 9:00

Ends at 10:00

Duration: 1 hours, 0 minutes

Location: home

Description: morning food
--------------------------------------------------------------------------------

Event name: lunch

Starts at 12:00

Ends at 13:00

Duration: 1 hours, 0 minutes

Location: home

Description: food
--------------------------------------------------------------------------------

Event name: meeting

Conflicts with: 
lunch

Starts at 12:00

Ends at 12:30

Duration: 0 hours, 30 minutes

Location: zoom

Description: while I'm eating food
--------------------------------------------------------------------------------

Event name: snack

Starts at 15:00

Ends at 16:00

Duration: 1 hours, 0 minutes

Location: home

Description: even more food
--------------------------------------------------------------------------------

Event name: dinner

Starts at 18:00

/Users/marivi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:161: UserWarning: You have set allow_conflicts=True.
The new event 'meeting' conflicts with the following 1 existing event(s):
lunch


Test the `delete_event()` method:

In [29]:
today.delete_event("meeting")

Are you sure that you want to delete the event named 'meeting'?
Enter 'yes' or 'no': yes


In [30]:
today.print_schedule()

Year: 2021, Month: 3, Day: 15
--------------------------------------------------------------------------------

Event name: breakfast

Starts at 9:00

Ends at 10:00

Duration: 1 hours, 0 minutes

Location: home

Description: morning food
--------------------------------------------------------------------------------

Event name: lunch

Starts at 12:00

Ends at 13:00

Duration: 1 hours, 0 minutes

Location: home

Description: food
--------------------------------------------------------------------------------

Event name: snack

Starts at 15:00

Ends at 16:00

Duration: 1 hours, 0 minutes

Location: home

Description: even more food
--------------------------------------------------------------------------------

Event name: dinner

Starts at 18:00

Ends at 19:00

Duration: 1 hours, 0 minutes

Location: home

Description: more food
--------------------------------------------------------------------------------

Length of day: 10.00 hours.
---------------------------------------------

In [12]:
# try again
today.delete_event("meeting")

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:191: UserWarning: The event named 'meeting' does not exist, so nothing further will be done.


In [13]:
# now, suppose we ask to delete the wrong event
today.delete_event("lunch")

Are you sure that you want to delete the event named 'lunch'?
Enter 'yes' or 'no': no
The event named 'lunch' will NOT be deleted.


In [14]:
today.print_schedule() # make sure we're still having lunch

Year: 2021, Month: 3, Day: 15
--------------------------------------------------------------------------------

Event name: breakfast

Starts at 9:00

Ends at 10:00

Duration: 1 hours, 0 minutes

Location: home

Description: morning food
--------------------------------------------------------------------------------

Event name: lunch

Starts at 12:00

Ends at 13:00

Duration: 1 hours, 0 minutes

Location: home

Description: food
--------------------------------------------------------------------------------

Event name: snack

Starts at 15:00

Ends at 16:00

Duration: 1 hours, 0 minutes

Location: home

Description: even more food
--------------------------------------------------------------------------------

Event name: dinner

Starts at 18:00

Ends at 19:00

Duration: 1 hours, 0 minutes

Location: home

Description: more food
--------------------------------------------------------------------------------

Length of day: 10.00 hours.
---------------------------------------------

Finally, see what happens when `allow_conflicts=False`:

In [31]:
# create a new day
tomorrow = Day(month=3, day=16, year=2021, allow_conflicts=False)

# add some events
tomorrow .add_event("lunch", time=12, duration=1, location="home", description="food")

tomorrow .add_event("meeting", time=12, duration=0.5, location="zoom", description="while I'm eating food")

RuntimeError: You have set allow_conflicts=False.
The new event 'meeting' conflicts with the following 1 existing event(s):
lunch

---
#### The `Week` class

Finally, add a `Week` class.

For the `__init()__` function, the arguments are:

- `month_start`: the month of the first day (currently must be Sunday) of your week.
    - Can be an `int` or a `str`: For example, `month_start=1` or `month_start="jan"` or `month_start="January"` are all equivalent.
    
    
- `day_start`: `int`, the calendar date of the first day in the week


- `year_start`: `int`, the year of the first day in the week


- `allow_conflicts`: same as for the `Day` class.



The main functions are the same as for `Day`:


- `add_event()`: same as for `Day`, but takes an additional argument `day_of_week`, which is a string giving the name of the day of the week ("mon" or "Monday", etc.)


- `delete_event()`:  same as for `Day`, and also takes an additional argument `day_of_week`.


- `print_schedule()`: same as for `Day`, but prints a schedule for each day in the week.

In [23]:
class Week(object):
    """Plan a full calendar week."""
    
    def __init__(self, month_start, day_start, year_start, allow_conflicts=True):
        """Begin planning your schedule for the week.

        Parameters
        ----------
        month_start : int or str
            The month of the first day in the week. 
            If int, give the number of the month (for example, `month`=1)
            If str, give the name of the month (for example, `month`="january")
        day_start : int
            The date of the first day in the week.
        year_start : int
            The year of the first day in the week.
        allow_conflicts : bool, default=True
            If True, events that overlap are allowed, and a warning will be raised by Day.add_event() when
             trying to add the conflicting event.
            Otherwise, an error will be raised by Day.add_event() when adding the event.

        Raises
        ------
        TypeError
            If any of the arguments are of an incorrect type.
        ValueError
            If any of the arguments have the correct type, but are invalid.

        Notes
        -----
        The week starts on Sunday.

        Currently, leap years are not accounted for.
        """
        # -------- define lists of valid strings for days of week and months of year ------
        
        self.days = ['sun', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat']
        self.months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        self.month_idxs = range(1, len(self.months)+1) # 1 for january, 2 for february, etc.

        # create dict with key as the number of the month and the value as the number of days in month
        days_in_month_list = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]  # neglect leap years for now
        self.days_per_month = {} # dict containing the number of days for each month.
        for month, ndays in zip(self.month_idxs, days_in_month_list):
            self.days_per_month[month] = ndays

        # --------- check the values of the arguments -------------------------------------

        # for the month, need to check if user provided int or str. we want to store it as an int.
        if type(month_start) == str:
            month = month_start.lower()[:3] # get the first 3 letters in lowercase
            if month not in self.months:
                raise(ValueError("Invalid month_start: {}".format(month_start)))
            else:
                self.month_start = self.months.index(month) + 1 # add one b/c counting from 1, not 0
        elif type(month_start) == int:
            if (month_start < 1) or (month_start > 12): # not a real month
                raise(ValueError("Invalid month_start (must be between 1 and 12): {}".format(month_start)))
            else: # everything is ok
                self.month_start = month_start
        else: # invalid type
            raise(TypeError("Invalid month_start (must be an int or str): {}".format(month_start)))

        # for the day, need to check if user provided a day within the given month
        if type(day_start) is not int: # check the type
            raise(TypeError("Invalid day_start (must be an int): {}".format(day_start)))
        else: # check the value
            if (day_start > self.days_per_month[self.month_start]) or (day_start < 1):
                raise(ValueError("Invalid day_start (too large or too small): {}".format(day_start)))
            else: # everything is ok
                self.day_start = day_start

        # check the year - just make sure it's an int
        if type(year_start) is not int:
            raise(TypeError("Invalid year_start (must be an int): {}".format(year_start)))
        else:
            self.year_start = year_start        

        # ------------------- initialize the week -------------------------------------
        
        # define a dictionary to hold the schedule for each day.
        # the keys will be the name of the days, 
        # and the values will be an instance of the Day class for the specified date.
        self.week = {}
        # keep track of the month, datte, and year for each day of the week
        current_month = self.month_start
        current_day = self.day_start
        current_year = self.year_start
        # add a Day object for each day of the week
        for day_name in self.days:
            # check if the current_day is within the current_month
            if (current_day > self.days_per_month[current_month]):
                current_day = 1 # start next month
                if current_month == 12:
                    # we're in the next year also
                    current_month = 1
                    current_year += 1
                else:
                    # still in same year, but next month
                    current_month += 1
            # create the Day and add it to the week
            self.week[day_name] = Day(month=current_month, day=current_day, year=current_year)
        
        
    def add_event(self, day_of_week, name, time=9, location=None, duration=1, description=None):
        """Add an event to a day in the week.
        
        Parameters
        ----------
        day_of_week : str
            The day of the week ('monday' or 'mon' or 'Monday', etc.)
        name : str
            A name for the event.
        time : int or float
            The time at which the event begins, in hours from midnight.
        location : str, default=None
            The location of the event. If not provided, `location` is not used.
        duration : int or float
            The duration of the event, in hours.
        description: str, default=None
            A description of the event. If not provided, `description` is not used.
            
        Raises
        ------
        ValueError
            If `day_of_week` is not a valid name of a day.
        """
        # check for a valid day
        day = day_of_week.lower()[:3] # get first three letters
        if day not in self.days:
            raise(ValueError("Invalid day_of_week: {}".format(day_of_week)))
        
        # add the event
        self.week[day].add_event(name, time=time, location=location, duration=duration, description=description)
        
    
    def delete_event(self, day_of_week, name, confirm=True):
        """Delete an event on the given day of the week.
        
        Parameters
        ----------
        day_of_week : str
            The day of the week ('monday' or 'mon' or 'Monday', etc.)
        name : str
            A name for the event.
        confirm : bool, default=True
            If True, ask the user if they are sure they would like 
            to delete the event called `name` before proceeding.
        """
        # check for a valid day
        day = day_of_week.lower()[:3] # get first three letters
        if day not in self.days:
            raise(ValueError("Invalid day_of_week: {}".format(day_of_week)))
            
        # delete the event
        self.week[day].delete_event(name, confirm=confirm)
        
    
    def print_schedule(self):
        """Prints the schedule for the whole week."""
        # print out a header for each day containinng day name, month, date, year
        header = "{}: {} {}, {}"
        divider = "-"*80
        
        # all of this is just formatting
        for day in self.days:
            # print out month, date, and year
            date_month = self.week[day].month # the number of the month
            date_month_str = self.months[date_month - 1] # the name of the month
            date_day = self.week[day].day
            date_year = self.week[day].year
            # format everything nicely
            print("\n")
            print(divider)
            print(divider)
            #print(header.format(day.upper(), date_month_str, date_day, date_year))
            header_for_today = header.format(day.upper(), date_month_str, date_day, date_year)
            print("{:^80s}".format(header_for_today))
            print(divider)
            self.week[day].print_schedule(print_day=False) # print out schedule for that day
            print(divider)

Test the `Week` class by adding or removing events and viewing the schedule:

In [24]:
my_week = Week('march', 15, 2021)
all_days = my_week.days[:]
week_days = my_week.days[1:-1] # mon - fri

In [25]:
# add lunch
for day in all_days:
    my_week.add_event(day, "lunch", time=12, location="home", 
                      description="lunch time!", duration=1)

# add work
for day in week_days:
    my_week.add_event(day, "work", time=9, duration=8, 
                      location="home", description="so I can buy lunch.")

# add PHY546
my_week.add_event('monday', 'PHY546', time=16.5, duration=1.25, 
                  location="zoom", description="fun with Python!")

# add other classes
for day in ['tues', 'thurs']:
    my_week.add_event(day, "PHY522", time=13.25, duration=1.33, 
                      location="Frey hall", description="learn some cool stuff about the ISM")
for day in ['mon', 'wed', 'fri']:
    my_week.add_event(day, 'PHY504', time=9.25, duration=1,
                     location="zoom", description="fun with Fortran!")
    
# add lab and office hours
my_week.add_event('tuesday', 'lab', time=18.5, duration=3, 
                  location="Physics building", description="fun with astronomy!")
my_week.add_event('friday', 'office hours', time=12, duration=1, 
                  location="zoom", description="maybe someone will show up!")

# add a meeting
my_week.add_event('thursday', 'group meeting', time=13.5, duration=1.33,  
                 location="zoom", description="let's get some work done.")



/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:161: UserWarning: You have set allow_conflicts=True.
The new event 'work' conflicts with the following 1 existing event(s):
lunch
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:161: UserWarning: You have set allow_conflicts=True.
The new event 'PHY546' conflicts with the following 1 existing event(s):
work
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:161: UserWarning: You have set allow_conflicts=True.
The new event 'PHY522' conflicts with the following 1 existing event(s):
work
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:161: UserWarning: You have set allow_conflicts=True.
The new event 'PHY504' conflicts with the following 1 existing event(s):
work
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:161: UserWarning: You have set allow_conflicts=True.
The new event 'office hours' conflicts with the following 1 existing event(s):
work
/anaconda/lib/python2.7/site-packages/ip

In [26]:
my_week.print_schedule()



--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                               SUN: mar 15, 2021                                
--------------------------------------------------------------------------------

Event name: lunch

Starts at 12:00

Ends at 13:00

Duration: 1 hours, 0 minutes

Location: home

Description: lunch time!
--------------------------------------------------------------------------------

Length of day: 1.00 hours.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                               MON: mar 15, 2021                                
-----------------------------------

In [20]:
# finally, let's take the week off from work.
for day in week_days:
    my_week.delete_event(day, 'work', confirm=False) 
    
# check the schedule again
my_week.print_schedule()



--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                               SUN: mar 15, 2021                                
--------------------------------------------------------------------------------

Event name: lunch

Starts at 12:00

Ends at 13:00

Duration: 1 hours, 0 minutes

Location: home

Description: lunch time!
--------------------------------------------------------------------------------

Length of day: 1.00 hours.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                               MON: mar 15, 2021                                
-----------------------------------

-----
### My to-do list

These are some ideas I had, but I think I should stop here...

- Add an option to format the time in 12-hour format.
    - Seems simple.


- Add an option to allow an event that spans multiple days (_i.e._, begins on Monday at noon and ends on Tuesday at noon).
    - Seems not quite as simple, but certainly possible.
    

- Handle leap years.
    - Should be simple.